Esse códgo tem como objetivo a montagem de um dataset para ser utilizado no trabalho de conclusão da terceira turma do curso do MBA da USP de IA e Big Data do ano de 2023/3024.

Baixando imagens estampadas dos sites:

https://bellaistampa.com.br/estampas-exclusivas?page=12

**Instalação das Bibliotecas Necessárias**

In [ ]:
pip install requests beautifulsoup4

**Código Python para Extrair e Baixar Imagens**

Função para baixar uma imagem da URL fornecida e salvar no caminho especificado.

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time

def download_image(url, save_path):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Lança um erro para respostas inválidas
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"Imagem salva em {save_path}")
    except requests.RequestException as e:
        print(f"Erro ao baixar a imagem: {e}")

Extrai links de imagens de uma URL e baixa todas as imagens encontradas.

In [ ]:
def extract_and_download_images(url, folder='downloaded_images', keyword='estampas'):
    try:
        # Faz a requisição para o site
        response = requests.get(url)
        response.raise_for_status()  # Verifica se a requisição foi bem-sucedida

        # Analisa o HTML da página
        soup = BeautifulSoup(response.text, 'html.parser')

        # Cria um diretório para as imagens, se não existir
        if not os.path.exists(folder):
            os.makedirs(folder)

        # Encontra todas as tags <img> na página
        #images = soup.find_all('img')
        images = [img for img in soup.find_all('img') if img.get('src') and keyword in img['src']]
        for img in images:
            # Extrai a URL da imagem
            img_url = img.get('src')
            if img_url and 'thumb_'not in img_url:
                # Pode haver URLs relativas, então precisamos completá-las
                complete_url = requests.compat.urljoin(url, img_url)
                file_name = os.path.join(folder, img_url.split('/')[-1])
                # Baixa a imagem
                download_image(complete_url, file_name)
            else:
                print(f"Ignorando imagem não iniciada por '{keyword}': {img_url}")
    except requests.RequestException as e:
        print(f"Erro ao acessar a página: {e}")


Testando script para baixar a imagem que será utilizada no dataset

In [ ]:
# Exemplo de uso
#rl = 'https://bellaistampa.com.br/estampas-exclusivas/viscose-super-soft-abacaxi-brasil-p'
#extract_and_download_images(url)

Imagem salva em downloaded_images/estampas-exclusivas-viscose-super-soft-abacaxi-brasil-p-1688491255511.jpg
Ignorando imagem não iniciada por 'estampas': https://76555.cdn.simplo7.net/static/76555/sku/thumb_estampas-exclusivas-viscose-super-soft-abacaxi-brasil-p-1688491255511.jpg
Ignorando imagem não iniciada por 'estampas': https://76555.cdn.simplo7.net/static/76555/sku/thumb_estampas-exclusivas-viscose-super-soft-abacaxi-brasil-p-1688491255511.jpg
Ignorando imagem não iniciada por 'estampas': https://76555.cdn.simplo7.net/static/76555/sku/thumb_estampas-exclusivas-viscose-super-soft-abacaxi-brasil-p-1688491255511.jpg
Ignorando imagem não iniciada por 'estampas': https://76555.cdn.simplo7.net/static/76555/sku/thumb_estampas-exclusivas-viscose-super-soft-abacaxi-brasil-p-1688491255511.jpg


**Extrai URLs dos detalhes do produto da página web que lista produtos.**


In [ ]:
def get_product_details_urls(url):
    try:
        # Faz a requisição para o site
        response = requests.get(url)
        response.raise_for_status()  # Verifica se a requisição foi bem-sucedida

        # Analisa o HTML da página
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontra todos os elementos que têm a classe 'product-list-item'
        product_items = soup.find_all(class_='product-list-item')

        # Lista para armazenar as URLs de detalhes do produto
        product_details_urls = []

        # Extrai a URL de cada produto
        for item in product_items:
            # Busca por um link dentro de cada item do produto
            detail_link = item.find('a', href=True)
            if detail_link:
                # Pode haver URLs relativas, então precisamos completá-las
                complete_url = requests.compat.urljoin(url, detail_link['href'])
                product_details_urls.append(complete_url)

        return product_details_urls

    except requests.RequestException as e:
        print(f"Erro ao acessar a página: {e}")
        return []

In [ ]:
# Configuração inicial do URL e iteração pelas páginas
base_url = 'https://bellaistampa.com.br/estampas-exclusivas?page='
for page_number in range(1, 13):  # Loop de página 1 até 12
    url = f"{base_url}{page_number}"
    product_urls = get_product_details_urls(url)
    for product_url in product_urls:
        print(product_url)
        extract_and_download_images(product_url)
        time.sleep(1)  # Pausa a execução por 1 segundo

https://bellaistampa.com.br/estampas-exclusivas/viscose-super-soft-abacaxi-brasil-p
Imagem salva em downloaded_images/estampas-exclusivas-viscose-super-soft-abacaxi-brasil-p-1688491255511.jpg
Ignorando imagem não iniciada por 'estampas': https://76555.cdn.simplo7.net/static/76555/sku/thumb_estampas-exclusivas-viscose-super-soft-abacaxi-brasil-p-1688491255511.jpg
Ignorando imagem não iniciada por 'estampas': https://76555.cdn.simplo7.net/static/76555/sku/thumb_estampas-exclusivas-viscose-super-soft-abacaxi-brasil-p-1688491255511.jpg
Ignorando imagem não iniciada por 'estampas': https://76555.cdn.simplo7.net/static/76555/sku/thumb_estampas-exclusivas-viscose-super-soft-abacaxi-brasil-p-1688491255511.jpg
Ignorando imagem não iniciada por 'estampas': https://76555.cdn.simplo7.net/static/76555/sku/thumb_estampas-exclusivas-viscose-super-soft-abacaxi-brasil-p-1688491255511.jpg
https://bellaistampa.com.br/estampas-exclusivas/viscocrepe-cosmos-p
Imagem salva em downloaded_images/estampas-exclu

**Zipa o conteúdo da pasta downloaded_images e as imagens.**

In [ ]:
import zipfile
import os

def zipdir(path, ziph):
    # Percorre todos os diretórios e arquivos
    for root, dirs, files in os.walk(path):
        for file in files:
            # Cria o caminho completo do arquivo
            full_path = os.path.join(root, file)
            # Adiciona o arquivo ao ZIP, especificando o caminho relativo
            ziph.write(full_path, os.path.relpath(full_path, os.path.join(path, '..')))

# Caminho do diretório que contém as imagens
dir_path = 'downloaded_images'

# Caminho do arquivo ZIP que será criado
zip_file_path = 'downloaded_images.zip'

# Criando o arquivo ZIP
with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir(dir_path, zipf)

print(f'Pasta {dir_path} foi compactada em {zip_file_path}')


Pasta downloaded_images foi compactada em downloaded_images.zip
